# Tugas 5: Menginvokasi model Bedrock untuk pembuatan kode

Dalam <i>notebook</i> ini, Anda mempelajari cara menggunakan model bahasa besar (LLM) untuk menghasilkan kode berdasarkan perintah teks.

Perintah yang digunakan dalam <i>notebook</i> ini adalah perintah <i>zero-shot </i>karena kami tidak memberikan contoh teks selain perintah itu sendiri.

Untuk mendemonstrasikan kemampuan pembuatan kode di Amazon Bedrock, mari melihat kasus penggunaan pembuatan email. Anda mempelajari cara menggunakan klien Boto3 untuk berkomunikasi dengan API Amazon Bedrock dan memberi API input yang terdiri dari tugas, instruksi, dan input untuk model di bagian kap untuk menghasilkan <i>output</i> tanpa memberikan contoh tambahan. Tujuannya adalah mendemonstrasikan bagaimana LLM yang canggih bisa dengan mudah memahami tugas yang ada dan menghasilkan <i>output</i> yang menarik.


## Skenario

Anda adalah Moe, seorang Analis Data di AnyCompany. Perusahaan ingin memahami kinerja penjualannya untuk berbagai produk selama setahun terakhir. Anda diberi set data bernama sales.csv. Set data berisi kolom berikut:

- Format tanggal (YYYY-MM-DD)
- Product_ID (pengidentifikasi unik untuk setiap produk)
- Harga (harga penjualan setiap produk)


Di notebook ini, Anda akan mempelajari cara menghasilkan kode untuk perintah yang diberikan. Anda memakai Meta LLama 3 yang menggunakan API Amazon Bedrock dengan klien Boto3.

## Tugas 5.1: Pengaturan lingkungan

Dalam tugas ini, Anda mengatur lingkungan Anda.


In [ ]:
#create a service client by name using the default session.
import json
import os
import sys

import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
bedrock_client = boto3.client('bedrock-runtime',region_name=os.environ.get("AWS_DEFAULT_REGION", None))


## Tugas 5.2: Pembuatan Kode

Dalam tugas ini, Anda menyiapkan input untuk layanan Amazon Bedrock guna menghasilkan program python untuk kasus penggunaan Anda.

#### Penyiapan lab - buat contoh data sales.csv untuk lab ini.



In [ ]:
# create sales.csv file
import csv

data = [
    ["date", "product_id", "price", "units_sold"],
    ["2023-01-01", "P001", 50, 20],
    ["2023-01-02", "P002", 60, 15],
    ["2023-01-03", "P001", 50, 18],
    ["2023-01-04", "P003", 70, 30],
    ["2023-01-05", "P001", 50, 25],
    ["2023-01-06", "P002", 60, 22],
    ["2023-01-07", "P003", 70, 24],
    ["2023-01-08", "P001", 50, 28],
    ["2023-01-09", "P002", 60, 17],
    ["2023-01-10", "P003", 70, 29],
    ["2023-02-11", "P001", 50, 23],
    ["2023-02-12", "P002", 60, 19],
    ["2023-02-13", "P001", 50, 21],
    ["2023-02-14", "P003", 70, 31],
    ["2023-03-15", "P001", 50, 26],
    ["2023-03-16", "P002", 60, 20],
    ["2023-03-17", "P003", 70, 33],
    ["2023-04-18", "P001", 50, 27],
    ["2023-04-19", "P002", 60, 18],
    ["2023-04-20", "P003", 70, 32],
    ["2023-04-21", "P001", 50, 22],
    ["2023-04-22", "P002", 60, 16],
    ["2023-04-23", "P003", 70, 34],
    ["2023-05-24", "P001", 50, 24],
    ["2023-05-25", "P002", 60, 21]
]

# Write data to sales.csv
with open('sales.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(data)

print("sales.csv has been created!")

## Tugas 5.3: Menganalisis penjualan dengan program Python yang dihasilkan Amazon Bedrock

In [ ]:
# define prompt template
from langchain_core.prompts import PromptTemplate

def format_prompt(actor:str, input:str):
    match actor:
        case "user":
            prompt_template =  """<|begin_of_text|><|start_header_id|>{actor}<|end_header_id|>\n\n{input}<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\n
"""
            prompt = PromptTemplate.from_template(prompt_template)
            return prompt.format(actor=actor,input=input)
        case _:
            print("requested actor >" + actor + "< is not supported")
            return ""    

In [ ]:
# Create the prompt
# Analyzing sales

prompt_data = """

You have a CSV, sales.csv, with columns:
- date (YYYY-MM-DD)
- product_id
- price
- units_sold

Create a python program to analyze the sales data from a CSV file. The program should be able to read the data, and determine below:

- Total revenue for the year
- Total revenue by product 
- The product with the highest revenue 
- The date with the highest revenue and the revenue achieved on that date
- Visualize monthly sales using a bar chart

Ensure the code is syntactically correct, bug-free, optimized, not span multiple lines unnessarily, and prefer to use standard libraries. Return only python code without any surrounding text, explanation or context.

"""
prompt=format_prompt("user",prompt_data)

In [ ]:
body = json.dumps({
    "prompt": prompt,
    "max_gen_len": 2048,
    "temperature": 0,
    "top_p": 1,
    }) 
    

## Tugas 5.4: Menginvokasi model

In [ ]:
modelId = "meta.llama3-8b-instruct-v1:0"
response = bedrock_client.invoke_model(body=body, modelId=modelId)
response_body = json.loads(response.get('body').read())
output_list = response_body.get("generation", [])
print(output_list)

##### <i aria-hidden="true" class="far fa-copy"></i> (Opsional) Salin kode yang dihasilkan dari <i>output</i> yang dicetak dan jalankan kode yang dihasilkan Bedrock pada sel di bawah ini untuk validasi.


Kini Anda telah bereksperimen menggunakan SDK `boto3` yang memberikan sedikit paparan mengenai API Amazon Bedrock. Dengan menggunakan API ini, Anda membuat program python untuk menganalisis dan memvisualisasikan data penjualan tertentu.



### Cobalah sendiri

- Ubah perintah untuk kasus penggunaan spesifik Anda dan evaluasi <i>output</i> model yang berbeda.
- Atur panjang token untuk memahami latensi dan responsivitas layanan.
- Terapkan berbagai prinsip rekayasa perintah untuk mendapatkan <i>output</i> yang lebih baik.

### Pembersihan

Anda telah menyelesaikan <i>notebook</i> ini. Untuk pindah ke bagian lab berikutnya, lakukan hal berikut:

- Tutup <i>file notebook</i> ini dan lanjutkan dengan **Tugas6**